In [0]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [0]:
!pip install eli5

In [0]:
pip install spacy && python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [0]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import spacy
import csv

def converteArquivo():
    with open('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_leis.txt', 'r') as in_file:
        nlp = spacy.load('pt')
        name = "Sentence #\tWord\tPOS\tTag\n"
        l = in_file.readlines()
    with open('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_leis.csv', 'w+') as file:
        file.write(name)
        seq = []
        index = 1
        for line in l:
            word = line.split()
            if len(word) != 2:
                sentences = ' '.join([s[0] for s in seq])
                doc = nlp(sentences)
                i = 0
                for s in doc:
                    if s.text == seq[i][0]:
                       file.write("Sentence: %d\t" % index)
                       file.write("%s\t" % seq[i][0])
                       file.write("%s\t" % s.pos_)
                       file.write("%s\n" % seq[i][1])
                       i += 1
                seq = []
                index+=1
            else:
                 seq.append((word[0], word[1]))

In [0]:
converteArquivo()

In [0]:
#Reading the dataset ptbr_leis 
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_leis.csv', encoding = 'utf-8', sep='\t')
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,EMENTA,VERB,O
1,Sentence: 1,:,PUNCT,O
2,Sentence: 1,APELAÇÃO,PROPN,O
3,Sentence: 1,CÍVEL,PROPN,O
4,Sentence: 1,-,PUNCT,O
5,Sentence: 1,AÇÃO,PROPN,O
6,Sentence: 1,DE,ADP,O
7,Sentence: 1,INDENIZAÇÃO,PROPN,O
8,Sentence: 1,POR,ADP,O
9,Sentence: 1,DANOS,NOUN,O


In [0]:
df.describe()

,Sentence #,Word,POS,Tag
count,137257,137211,137257,137257
unique,7732,13742,15,13
top,Sentence: 5912,",",NOUN,O
freq,204,9794,25321,119188


In [0]:
#Exibe as tags únicas do dataset de leis
df['Tag'].unique()

array(['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-LEGISLACAO',
       'I-LEGISLACAO', 'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA', 'B-PESSOA',
       'I-PESSOA', 'B-TEMPO', 'B-LOCAL', 'I-TEMPO', 'I-LOCAL'],
      dtype=object)

In [0]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-JURISPRUDENCIA,679
1,B-LEGISLACAO,1235
2,B-LOCAL,272
3,B-ORGANIZACAO,1375
4,B-PESSOA,1030
5,B-TEMPO,789
6,I-JURISPRUDENCIA,1611
7,I-LEGISLACAO,5619
8,I-LOCAL,374
9,I-ORGANIZACAO,2324


In [0]:
#Checking null values, if any.
df.isnull().sum()

Sentence #     0
Word          46
POS            0
Tag            0
dtype: int64

In [0]:
df = df.fillna(method = 'ffill')

In [0]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET PARQUET INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA .'

In [0]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('EMENTA', 'VERB', 'O'), (':', 'PUNCT', 'O'), ('APELAÇÃO', 'PROPN', 'O'), ('CÍVEL', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('AÇÃO', 'PROPN', 'O'), ('DE', 'ADP', 'O'), ('INDENIZAÇÃO', 'PROPN', 'O'), ('POR', 'ADP', 'O'), ('DANOS', 'NOUN', 'O'), ('MORAIS', 'ADP', 'O'), ('-', 'PUNCT', 'O'), ('PRELIMINAR', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('ARGUIDA', 'PROPN', 'O'), ('PELO', 'VERB', 'O'), ('MINISTÉRIO', 'PROPN', 'B-ORGANIZACAO'), ('PÚBLICO', 'PROPN', 'I-ORGANIZACAO'), ('EM', 'ADP', 'O'), ('GRAU', 'PROPN', 'O'), ('RECURSAL', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('NULIDADE', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('AUSÊNCIA', 'PROPN', 'O'), ('DE', 'ADP', 'O'), ('INTERVENÇÃO', 'PROPN', 'O'), ('DO', 'PROPN', 'O'), ('PARQUET', 'PROPN', 'O'), ('PARQUET', 'PROPN', 'O'), ('INSTÂNCIA', 'PROPN', 'O'), ('A', 'DET', 'O'), ('QUO', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('PRESENÇA', 'PROPN', 'O'), ('DE', 'ADP', 'O'), ('INCAPAZ', 'PROPN', 'O'), ('-', 'PUNCT', 'O'), ('PREJUÍZO', 'PROPN', 'O'), ('EXISTENTE', 'PROP

In [0]:
sentences = getter.sentences

In [0]:
def isArtigo(word):
  art = ["art","artigo","ART","ARTIGO"]
  for l in word:
    if l in art:
       return True
    else:
       return False

def isNumRomano(word):
  romanos = ["I", "V", "X"]  #numeros romanos existentes no dataset
  num = word.upper()
  for l in num:
    if l in romanos:
      return True
    else:
      return False 

def isSymbol(word):
  symbols = ["-","(",")",":","%","$","/","§","nº","º","ª","°"]
  for l in word:
    if l in symbols:
       return True
    else:
       return False

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'isArtigo()': isArtigo(word),
        'isNumRomano()':isNumRomano(word),
        'isSymbol()':isSymbol(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            'isArtigo': isArtigo(word),
            'isNumRomano()':isNumRomano(word),
            'isSymbol()':isSymbol(word),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],

        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            'isArtigo': isArtigo(word),
            'isNumRomano()':isNumRomano(word),
            'isSymbol()':isSymbol(word),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],

        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [0]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [0]:
import eli5
eli5.show_weights(crf, top=10)

From \ To,O,B-JURISPRUDENCIA,I-JURISPRUDENCIA,B-LEGISLACAO,I-LEGISLACAO,B-LOCAL,I-LOCAL,B-ORGANIZACAO,I-ORGANIZACAO,B-PESSOA,I-PESSOA,B-TEMPO,I-TEMPO
O,4.55,2.592,0.0,2.682,0.0,3.461,0.0,3.228,0.0,3.474,0.0,6.011,0.0
B-JURISPRUDENCIA,-2.225,0.0,6.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-JURISPRUDENCIA,-1.456,0.0,6.634,0.0,0.0,0.83,0.0,-2.755,0.0,0.0,0.0,0.0,0.0
B-LEGISLACAO,-2.397,0.0,0.0,0.0,5.997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-LEGISLACAO,-1.695,0.0,0.0,-3.284,6.752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-LOCAL,-0.969,0.0,0.0,0.0,0.0,0.0,6.77,0.0,0.0,-0.0,0.0,0.0,0.0
I-LOCAL,-0.539,0.0,0.0,0.0,0.0,0.0,6.709,0.244,0.0,0.0,0.0,0.0,0.0
B-ORGANIZACAO,-0.336,0.0,0.0,0.0,0.0,0.107,0.0,-0.096,6.645,0.0,0.0,0.0,0.0
I-ORGANIZACAO,-0.596,0.158,0.0,0.0,0.0,0.0,0.0,0.001,7.015,0.0,0.0,1.034,0.0
B-PESSOA,-1.449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.591,0.0,0.0


In [0]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9847088625463212


In [0]:
report = flat_classification_report(y_test, y_pred)
print(report)

                  precision    recall  f1-score   support

B-JURISPRUDENCIA       0.93      0.79      0.85       127
    B-LEGISLACAO       0.93      0.90      0.92       265
         B-LOCAL       0.87      0.77      0.81        43
   B-ORGANIZACAO       0.92      0.89      0.91       260
        B-PESSOA       0.96      0.91      0.93       213
         B-TEMPO       0.94      0.88      0.91       165
I-JURISPRUDENCIA       0.93      0.84      0.89       301
    I-LEGISLACAO       0.97      0.94      0.96      1244
         I-LOCAL       0.87      0.88      0.87        51
   I-ORGANIZACAO       0.94      0.92      0.93       456
        I-PESSOA       0.98      0.95      0.96       431
         I-TEMPO       0.93      0.91      0.92       139
               O       0.99      1.00      0.99     23912

        accuracy                           0.98     27607
       macro avg       0.94      0.89      0.91     27607
    weighted avg       0.98      0.98      0.98     27607

